In [3]:
#Regression 실습 전 필요 항목들 import
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import sklearn
from tqdm import tqdm

0. 데이터셋 재구성하기(Train/test set)</br>
외부에서 데이터 셋을 가져오는 부분이며, 이번 예제의 경우 워낙 유명한 예제이니</br>
인터넷에 pytorch예제를 잘 수행할 수 있도록 어느정도 정제가 되어 있음</br>
향후 전처리 방법에 대해 숙지하기

In [6]:
# raw data load & convert to numpy

data_url = "http://lib.stat.cmu.edu/datasets/boston" #예제에 사용할 데이터넷
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
data = pd.DataFrame(data, columns= ["CRIM", "ZN", "iNDUS", "CHAS", "NOX", "RM", "AGE"
                                     , "DIS", "RAD", "TAX", "RTRATIO", "B", "LSTAT"])
target = raw_df.values[1::2, 2]
target = pd.DataFrame(target, columns=["MEDV"])

In [7]:
#전처리된 데이터가 잘 전처리 되었는지 확인해보기
data.head()

,CRIM,ZN,iNDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,RTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [8]:
target.head()

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [10]:
print(data.shape, target.shape)

(506, 13) (506, 1)


In [11]:
#데이터 전처리 (input, output)가 잘 됬음을 확인했으니 이제 학습을 위한 준비

from sklearn.model_selection import train_test_split

#데이터를 나눠주는 작업 수행 -> 학습을 위해 데이터를 분절함
X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.2, random_state=42)
#학습용 데이터 / 평가용 데이터를 나누는데 Test_size가 0.2라는 것은 8:2비율로 나누라는 뜻임
#random_stat는 데이터 분할 때 좀 섞어서 분할해라? 라는 뜻임

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(404, 13) (102, 13) (404, 1) (102, 1)


In [12]:
#데이터 스케일링 작업수행
#이거는 학습에 사용되는 데이터의 숫자가 어떤건 0....0으로 되있고 어떤건 100~200으로
#변하는 수치가 다 제각각인데 이걸 그냥 학습 시키면 숫자가 큰 데이터 값에
#의존하는 경향성이 높아짐. 따라서 데이터의 col별로 0~1사이에서만 움직이는 데이터셋으로
#전부 스케일을 맞춰준다 보면 된다.
#스케일 과정은 표준 정규분포로 만든다 보면 된다.(평균, 분산을 사용함)

from sklearn.preprocessing import StandardScaler #정규화 과정을 해주는 모듈

#데이터 스케일링 작업
scaler = StandardScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) #정규화 모델의 기준은 학습용 데이터로만 fit함

1. 고전적 추론기 사용 -> LSE 기반의 선형 회귀 모델</br>
이 모델의 구현은 sklearn Linear Regression Model을 활용함</br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [13]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression(fit_intercept=True)
LR_model.fit(X_train, Y_train)

print(LR_model.intercept_)#모델의 y절편
print(LR_model.coef_)#모델의 계수

[22.79653465]
[[-1.00213533  0.69626862  0.27806485  0.7187384  -2.0223194   3.14523956
  -0.17604788 -3.0819076   2.25140666 -1.76701378 -2.03775151  1.12956831
  -3.61165842]]


In [17]:
#위 LSE모델 해석
import statsmodels.api as sm #LR모델은 해석이 가능한데 그걸 해주는 모듈

X_train_sm = sm.add_constant(X_train) #y절편을 추가해주겠다는 뜻

LR_model_sm = sm.OLS(Y_train, X_train_sm).fit() #LSE(OLS)모델로 fit하겟다는 뜻

print(LR_model_sm.summary()) #분석한 모델에 대한 결과값 출력
#아래 결과값은 y = ax + b라는 선형추세선이 나왔을 때
#그 추세선에 대한 설명이 담긴 문구라 보면 된다.

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     90.43
Date:                Sun, 28 Apr 2024   Prob (F-statistic):          6.21e-109
Time:                        18:23:50   Log-Likelihood:                -1194.3
No. Observations:                 404   AIC:                             2417.
Df Residuals:                     390   BIC:                             2473.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.7965      0.236     96.774      0.0

2. torch를 활용한 딥러닝(Linear Layer)을 구현해보기</br>
정확히는 Fully Connected Layer(Linear Layer)을 모델링한다 보면 된다</br>
이거는 FC Layer을 이용한 회귀 모델 구현 이렇게 표현한다.

In [19]:
#바닐라 훈련 버전 만들기 (순정 버전 만들기라 보면 된다..)
#처음 명세서에서 레이어를 2개 만들며, (13,6) -> ReLU -> (6,1)이렇게 지나가게 만든다 정의함

class MyRegressionModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.hidden_size = input_size // 2 #13을 나누기 하고 버리면 6이 나오니까

        self.fc1 = nn.Linear(in_features=input_size, out_features=self.hidden_size)
        self.fc2 = nn.Linear(in_features=self.hidden_size, out_features=1)
        #fc1 레이어는 (13, 6), fc2레이어는 (6,1)을 갖게됨

    
    def forward(self, x): #레이어1 -> Relu -> 레이어2
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
############모델 설계 끝###############

ex_model = MyRegressionModel(13)

In [25]:
#loss Function 어떤 거 쓸지 정의하기, 최적화 방법론 어떤거 쓸지 정의하기
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(ex_model.parameters(), lr=0.01)

In [23]:
#앞에서 raw data를 최초로 정제한 데이터를 텐서 자료형으로 전처리 하기
import torch.utils


data_tensor = torch.tensor(np.array(data), dtype=torch.float32)
target_tensor = torch.tensor(np.array(target), dtype=torch.float32)
#맨 위에서 불러온 raw data를 텐서 자료형으로 만듬

dataset = torch.utils.data.TensorDataset(data_tensor, target_tensor)

#앞에 LSE 모델용 데이터 셋처럼 훈련/평가용 데이터셋 나누기(과정 같음)
test_size = 0.2
test_size = int(np.round(test_size * len(dataset)))

train_dataset, test_dataset = torch.utils.data.random_split(dataset, 
                                                            [len(dataset)-test_size, test_size])

#LSE모델에 사용한 데이터셋이랑 똑같게 훈련/평가용 데이터는 8:2로 나누고 셔플도 하지만
#딥러닝 할때는 텐서 자료형만 받으니 자료형만 다르다 보면 된다.


In [27]:
#훈련 과정 함수 만들기(100번 훈련함)

for epoch in tqdm(range(100)):
    for(x, y) in train_dataset:
        #Forward 과정 -> 전사 과정
        y_pred = ex_model(x) #레이어를 통과하니 예측값 Y^hat가 나옴

        #평가 과정(loss function으로 관측값 측정값 차이 얼마나는지)
        loss = loss_fn(y_pred, y)

        #Backward 과정 -> 평가 결과를 다시 레이어의 Weight matrix에 업데이트
        #여기는 백워드 과정에서 옵티마이저 초기화 부분이라 보면 된다.
        optimizer.zero_grad()
        loss.backward()

        #Weight matirx에 업데이트는 여기서 이뤄짐
        #Weight matirx의 원소값들은 model parameter이라 부른다
        optimizer.step()

100%|██████████| 100/100 [00:43<00:00,  2.27it/s]


위 학습과정은 CPU만 쓴거라 대용량 모델이 아님</br>
대용량 모델을 학습하기 위해 Batch Training 버전을 만들자

## Batch Training Version

In [28]:
#batch training은 그냥 dataloader 모듈 불러오면 끝임

from torch.utils.data import DataLoader

#batch size 결정하는 부분
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#배치 사이즈 10으로 정의하고 훈련(위에거랑 코드는 같음)
#배치사이즈를 10으로 정의하면 위에꺼 훈련은 한개씩만 데이터 훈련할거 여기서는 10개씩 쓴다는거임
#이게 병렬처리임(위에꺼는 병렬처리 아님...) 그래서 속도가 빨라질 거임 -> 근데 CPU라 안빠름 ㅋㅋ;;;

for epoch in tqdm(range(100)):
    for(x, y) in train_dataset:
        y_pred = ex_model(x) #Forward 과정 -> 전사 과정
        loss = loss_fn(y_pred, y) #평가 과정

        optimizer.zero_grad() #backward - 초기화
        loss.backward()

        optimizer.step() #backward - 모델 파라미터 업데이트

100%|██████████| 100/100 [00:41<00:00,  2.39it/s]


위 CPU 순정학습(한개씩 데이터 훈련) </br>
CPU 버전 병렬처리(batch training) 을 했으니 </br>
GPU를 사용한 병럴처리를 해보자

In [39]:
#GPU사용 가능 확인하기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#모델을 GPU로 이동
ex_model.to(device)

# 옵티마이저에 연결된 파라미터를 GPU로 이동
# for param in ex_model.parameters():
#     param.to(device)

# 훈련 및 평가 데이터셋을 생성한 후에 GPU로 이동
train_dataset = torch.utils.data.TensorDataset(data_tensor[:len(dataset)-test_size].to(device), 
                                               target_tensor[:len(dataset)-test_size].to(device))
test_dataset = torch.utils.data.TensorDataset(data_tensor[-test_size:].to(device), 
                                              target_tensor[-test_size:].to(device))

#여기서부터는 위 배치사이즈 설정한거랑 똑같은 코드
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in tqdm(range(100)):
    for(x, y) in train_dataset:
        y_pred = ex_model(x.to(device)) #Forward 과정 -> 전사 과정
        loss = loss_fn(y_pred, y.to(device)) #평가 과정

        optimizer.zero_grad() #backward - 초기화
        loss.backward()

        optimizer.step() #backward - 모델 파라미터 업데이트

cuda


100%|██████████| 100/100 [01:09<00:00,  1.44it/s]
